# fuzzy yona

In [1]:
import pandas as pd
from fuzzywuzzy import fuzz

pd.set_option('display.max_columns', None)  # Esto mostrará todas las columnas
pd.set_option('display.max_colwidth', None)  # Esto evitará que el contenido de las celdas se trunque

c:\Users\JMP\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
scrapeo_dia = pd.read_csv("../data/scrapeo(2023-07-10 14-50-32).csv", index_col = 0)
datamarket_dia = pd.read_csv("../data/datamarket-dia.csv", index_col = 0)

In [10]:
# paso a listas las cols de categoría del dataframe scrapeado y del inicial
cat_nuevas = scrapeo_dia['category'].unique().tolist()
cat_antiguas = datamarket_dia['category'].unique().tolist()

def cat_max(col, limpita):
    '''
    Devuelve categoría tras aplicar fuzzywuzzy
    '''

    maximo = 0

    for l in limpita:

        parecido = fuzz.ratio(col, l)

        if parecido > maximo:

            maximo = parecido
            
            categoria = l

    return categoria

# splitea los valores de las categorías scrapeadas por '_'
spliteada = []

for s in cat_nuevas:

    a = s.split('_')

    spliteada.append(a)

# flatea la lista para pasar de lista de listas a lista de strs
flat_list = []

for sublist in spliteada:

    for item in sublist:

        flat_list.append(item)

# apendea en una lista todas las palabras eliminando aquellas de poco interés para el fuzzywuzzy
limpita = []

for f in flat_list:

    # se evitan los nexos entre palabras
    if f == 'y' or f == 'e' or f == 'con':
        pass

    else:
        limpita.append(f)


# añade a lo lista limpia anterior algunas categorías especiales como navidad y otros términos complicados de clasificar
apendeos = ['gluten', 'lacteos', 'navidad', 'dieteticos', 'solidario', 'aceitunas', 'sal', 
            'cuidado', 'internacional', 'mermeladas', 'licores', 'sopas', 'espumosos']

for a in apendeos:
    
    limpita.append(a)

# splitea los valores de las categorías iniciales por '_'
aux =[]

for c in cat_antiguas:

    aux.append(c.split('_'))


# descarta palabras que dificultan el matcheo por fuzzywuzzy
aux2 = []
word = ''

descartes = ['y', 'e', 'vinagre', 'con', 'del', 'al', 'de', 'dia', 'alimentacion', 'fresco', 'frescos', 
            'desayuno', 'despensa', 'bodega', 'drogueria', 'desayunos', 'dulces',
            'cocina', 'personal', 'soy', 'bano', 'corporal', 'preparacion', 'bebidas']

for a in aux:

    for i in a:
        
        if i in descartes:
            pass

        else:
            word += i + ' '
    
    aux2.append(word.rstrip(' '))

    word = ''

# para cada una de las listas de strs de categorías iniciales tras los descartes se selecciona solo la primera palabra, la más representativa
aux3 = []

for a in aux2:

    b = a.split(' ')
    aux3.append(b[0])

# genera un df con la palabra representativa de las categorías iniciales y con las resultantes del fuzzywuzzy
df = pd.DataFrame()
df['antiguas'] = pd.DataFrame(aux3)
df['fuzzcat'] = df.apply(lambda x: cat_max(x['antiguas'], limpita), axis = 1)

# YA SE HAN RELACIONADO LAS CATEGORÍAS CON FUZZYWUZZY
# AHORA HAY QUE HACER EL PROCESO CONTRARIO DE RELACIONAR LAS PALABRAS CLAVES MATCHEADAS CON LOS NOMBRES DE LAS CATEGORÍAS

# se repite el proceso de descartar nexos entre palabras para las categorías scrapeadas, pero se mantiene la estructura de lista de listas y no se flatea como anteriormente
definitiva = []
aux = []
for sublist in spliteada:

    for f in sublist:

        if f == 'y' or f == 'e' or f == 'con':
            pass

        else:
            aux.append(f)
    definitiva.append(aux)
    aux =[]


def convertir_lista_a_diccionario(lista, claves):
    '''
    Convierte una lista de listas en un diccionario según unas keys dadas
    '''
    dictio = {}
    
    for i, sublist in enumerate(lista):

        clave = claves[i]
        dictio[clave] = sublist
    
    return dictio

# crea un diccionario {categorias scrapeadas}: [palabras clave de la categoría usadas para fuzzywuzzy]
# ejemplo {azucar_chocolates_y_caramelos}: [azucar, chocolates, caramelos]
lista_de_listas = definitiva
dictio = convertir_lista_a_diccionario(lista_de_listas, cat_nuevas)

# relaciona los apendeos con categorías existentes o crea nuevas categorías
# nuevas categorías:
dictio['gluten'] = ['gluten'] # sorry celiacos
dictio['solidario'] = ['solidario']
dictio['navidad'] = ['navidad']
dictio['internacional'] = ['internacional']

# relaciona términos con categorías existentes:
dictio['leche_huevos_y_mantequilla'].append('lacteos')
dictio['galletas_bollos_y_cereales'].append('dieteticos')
dictio['patatas_fritas_encurtidos_y_frutos_secos'].append('aceitunas')
dictio['aceites_salsas_y_especias'].append('sal')
dictio['perfumeria_higiene_salud'].append('cuidado')
dictio['azucar_chocolates_y_caramelos'].append('mermeladas')
dictio['cervezas_vinos_y_bebidas_con_alcohol'].append('licores')
dictio['conservas_caldos_y_cremas'].append('sopas')
dictio['cervezas_vinos_y_bebidas_con_alcohol'].append('espumosos')

# transforma las palabras clave de fuzzywuzzy en las categorías scrapeadas categorías scrapeadas
# ejemplo leche: leche_huevos_y_mantequilla
listiña = []

for i in df['fuzzcat']:

    for key, values in dictio.items():

        if i in values:

            listiña.append(key)

# añade al df
df['feas'] = cat_antiguas
df['transformadas'] = listiña

transformadas = df['transformadas'].tolist()
feas = df['feas'].unique().tolist()

# diccionario para transformas las categorías iniciales en las categorías scrapeadas
# {categoría inicial}: categoría scrapeada
hada = {}

for i in range(0, len(transformadas)):

    hada[feas[i]] = transformadas[i]

# map
datamarket_dia['nuevas_categorias'] = datamarket_dia['category'].map(hada)

datamarket_dia.sample(10)

,category,name,description,price,reference_price,reference_unit,insert_date,product_id,nuevas_categorias
supermarket,,,,,,,,,
dia-es,despensa_lacteos_y_huevos_huevos,GRUPO DE LUCAS huevos frescos maíz categoría A clase M/L estuche 12 uds,NaN,1.89,0.16,ud,2021-11-10 00:00:00,f10cd3c8ee0e6543a66f1894c1572bcb,leche_huevos_y_mantequilla
dia-es,despensa_desayunos_y_dulces_cacao_y_cremas_de_cacao,COLACAO cacao soluble instantáneo turbo bote 750 gr,NaN,5.39,7.19,kg,2022-02-11 00:00:00,7b038350d84882425d2bb71b62fb3db0,cafe_cacao_e_infusiones
dia-es,mascotas_otros_animales,AS alimento mix para roedores bolsa 1 Kg,NaN,2.19,2.19,kg,2022-05-04 00:00:00,7f34f4aa6bc6702950e13dff451ea15c,mascotas
dia-es,cuidado_personal_bano_e_higiene_personal_jabon,BONTE jabón líquido de manos almendra y miel dosificador 500 ml,NaN,1.19,2.38,l,2021-09-01 00:00:00,6cbca5bfbf2ed8771f74db32664e6ab8,perfumeria_higiene_salud
dia-es,congelados_helados,DIA TEMPTATION helado bombón chocolate almendrado caja 6 uds 540 gr,NaN,2.09,3.87,kg,2021-09-06 00:00:00,93b846ea0bbe246a9f957bfc03dfd6f4,congelados
dia-es,despensa_desayunos_y_dulces_chocolates_y_bombones,LINDT Lindor bombón chocolate blanco con fresa unidad (15 gr aprox.),NaN,0.49,32.99,kg,2021-11-21 00:00:00,9ac480936bfbe2e9d2331f379c2d8a95,azucar_chocolates_y_caramelos
dia-es,desayuno_dulces_y_pan_chocolates_y_bombones_tabletas,MILKA chocolate con leche y almendras tableta 125 gr,NaN,1.89,15.12,kg,2021-03-17 00:00:00,a19ac8007112c1c05cb245631a907eff,panes_harinas_y_masas
dia-es,despensa_aceites_vinagres_y_alinos_aceites,DCOOP aceite de oliva virgen extra botella 1 lt,NaN,5.89,5.89,l,2022-06-20 00:00:00,25a38ffe3f60e13ffe451b12400826e0,aceites_salsas_y_especias
dia-es,cuidado_personal_bano_e_higiene_personal_champus,BONTE champú familiar cabello normal botella 1 L,NaN,0.95,0.95,l,2021-06-10 00:00:00,a5b90a1b1a06b1cfe90b6debc83e0b55,perfumeria_higiene_salud
